In [1]:
%load_ext autoreload
from google.auth import default
credentials, project = default()

In [2]:

%autoreload 2
import os
import sys
from pathlib import Path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from dotenv import load_dotenv
load_dotenv()

from src.config_loader import config
from src.gcs_storage import read_from_gcs, upload_to_gcs,get_wiktionary_cache_path 
from src.utils import get_first_n_items
from src.wiktionary import generate_wiktionary_links
from tqdm import tqdm
print(config.TARGET_LANGUAGE_NAME)
COLLECTION = "WarmUp150"

German


# Wiktionary Links

```raw
These are stored as a space separated string of either the words or a <a href> <\a> item.

Within the phrase dictionary they are stored with the key 'wiktionary_link'

phrase_key : {'english' : phrase key,
            'french' : phrase key,
            'wiktionary_links' : <a href>phrase</a> key}```

We store the wiktionary links as a cache so they can be reused in, e.g. the stories

In [4]:
# get current translated JSON file
from src.gcs_storage import get_wiktionary_cache_path, get_translated_phrases_path

phrase_translations = read_from_gcs(bucket_name=config.GCS_PRIVATE_BUCKET, file_path=get_translated_phrases_path(collection=COLLECTION))

In [7]:
word_link_cache= read_from_gcs(config.GCS_PRIVATE_BUCKET, file_path = get_wiktionary_cache_path())

In [ ]:
from src.wiktionary import generate_wiktionary_links
from src.config_loader import config


# Find failed entries (where key == value)
failed_words = [word for word, link in word_link_cache.items() if word == link]
print(f"Found {len(failed_words)} failed entries to retry: {failed_words}")

# Retry each failed word using the updated function
updated_cache = word_link_cache.copy()
successful_retries = []

for word in failed_words:
    # Use the generate_wiktionary_links function to retry this word
    _, updated_cache = generate_wiktionary_links(
        phrase=word,
        language_name=config.TARGET_LANGUAGE_NAME,  # or specify "German" directly
        word_link_cache={},
        return_cache=True
    )
    
    # Check if we got a successful link this time
    try:
        if updated_cache[word] != word:
            successful_retries.append(word)
            print(f"✅ Successfully found link for '{word}': {updated_cache[word]}")
        else:
            print(f"❌ Still no link found for '{word}'")
    except KeyError:
        print(f"Key error with {word}")
        continue

print(f"\nSummary:")
print(f"Retried {len(failed_words)} words")
print(f"Successfully found links for {len(successful_retries)} words")
print(f"Updated cache size: {len(updated_cache)}")

# Your updated cache is now ready
word_link_cache = updated_cache

Found 906 failed entries to retry: ['Arbeit', 'Hilfe', '-', 'Auto', 'Glas', 'Urlaub', 'Meer', 'E-Mail', 'Feierabend', 'Frühjahr', 'Abschluss', 'Projekt', 'Freitag', 'Herd', 'Tochter', 'Flughafen', 'Idee', 'Fehler', 'Geburtstagsfeier', 'Konzert', 'Schluss', 'Zug', 'Auszeichnung', 'Überraschungsparty', 'Mama', 'weitergekocht', 'Zukunft', 'Seite', 'Teilnehmer', 'Restaurant', 'Monat', 'Wohnung', 'Gedanken', 'Party', 'Gästen', 'Kaffee', 'Methode', 'Firma', 'Mantel', 'Garderobe', 'Salz', 'Geschmack', 'Kuchen', 'Ok', 'Liste', 'Auftrag', 'Woche', 'Team', 'Geister', 'Phänomene', 'Bett', 'Formular', 'Bruder', 'Vogel', 'Gerücht', 'Gitarre', 'Lust', 'Campingausflug', 'Gewohnheiten', 'Lotto', 'Flasche', 'Wahnsinn', 'Aussicht', 'Schlüssel', 'Wochenende', 'Tasche', 'Lösung', 'Radio', 'Chaos', 'Ordnung', 'Blumen', 'Bericht', 'Nase', 'Tür', 'Stadt', 'Regens', 'Ideen', 'Möglichkeiten', 'Pfund', 'Gott', 'Zeug', 'Show', 'Westküste', 'Suppe', 'Norden', 'Reiseabenteuern', 'Satz', 'Hausaufgaben', 'Helm', 'Ma

KeyError: 'E-Mail'

In [ ]:
from src.wiktionary import add_wiktionary_links
phrase_translations, word_link_cache = add_wiktionary_links(phrase_translations, word_link_cache, overwrite=False)

In [ ]:
upload_to_gcs(obj=phrase_translations, bucket_name=config.GCS_PRIVATE_BUCKET, file_name=get_translated_phrases_path(collection=COLLECTION))

In [ ]:
upload_to_gcs(word_link_cache, bucket_name=config.GCS_PRIVATE_BUCKET, file_name=get_wiktionary_cache_path())